# T1530 - Data from Cloud Storage Object
Adversaries may access data objects from improperly secured cloud storage.

Many cloud service providers offer solutions for online data storage such as Amazon S3, Azure Storage, and Google Cloud Storage. These solutions differ from other storage solutions (such as SQL or Elasticsearch) in that there is no overarching application. Data from these solutions can be retrieved directly using the cloud provider's APIs. Solution providers typically offer security guides to help end users configure systems.(Citation: Amazon S3 Security, 2019)(Citation: Microsoft Azure Storage Security, 2019)(Citation: Google Cloud Storage Best Practices, 2019)

Misconfiguration by end users is a common problem. There have been numerous incidents where cloud storage has been improperly secured (typically by unintentionally allowing public access by unauthenticated users or overly-broad access by all users), allowing open access to credit cards, personally identifiable information, medical records, and other sensitive information.(Citation: Trend Micro S3 Exposed PII, 2017)(Citation: Wired Magecart S3 Buckets, 2019)(Citation: HIPAA Journal S3 Breach, 2017) Adversaries may also obtain leaked credentials in source repositories, logs, or other means as a way to gain access to cloud storage objects that have access permission controls.

## Atomic Tests:
Currently, no tests are available for this technique.

## Detection
Monitor for unusual queries to the cloud provider's storage service. Activity originating from unexpected sources may indicate improper permissions are set that is allowing access to data. Additionally, detecting failed attempts by a user for a certain object, followed by escalation of privileges by the same user, and access to the same object may be an indication of suspicious activity.

## Shield Active Defense
### Pocket Litter 
 Place data on a system to reinforce the legitimacy of the system or user. 

 Pocket Litter is data placed on a system to convince an adversary that the system and users are real.  Pocket litter includes documents, registry entries, log history, browsing history, connection history, and other user data that one would expect to exist on a user's computer.  This content may overlap with Decoy Content, however Pocket Litter covers aspects beyond just content (e.g.: Installed Applications, source code, clutter on a system, etc.).
#### Opportunity
In an adversary engagement scenario, there is an opportunity to seed content to influence an adversary's behaviors, test their interest in specific topics, or add legitimacy to a system or environment.
#### Use Case
A defender can stage a variety of pocket litter files on an attached storage space. This data may include topics that align to a persona, topics an adversary is interested in, etc.
#### Procedures
When staging a decoy system and user account, populate a user's folders and web history to make it look realistic to an adversary.
Stage a USB device with documents on a specific topic in order to see if they are exfiltrated by an adversary.